In [26]:
#####################   PART 1  ########################

!pip install pandasql
import pandasql as ps
import tensorflow as tf
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.datasets import imdb
import tensorflow as tf


You should consider upgrading via the 'pip install --upgrade pip' command.


In [95]:
#Dataset load and prepare input and target labels, make train test split
X_Y_full = pd.read_csv('Dataset.csv')
print(len(X_Y_full))
#X_Y_full = ps.sqldf("select * from X_Y_full order by review desc limit 20000")
X_Y_full['review']=X_Y_full['review'].str.replace('<br />', '').replace('[^a-zA-Z]', '')
train, validate, test = np.split(X_Y_full.sample(frac=1), [int(.5*len(X_Y_full)), int(.8*len(X_Y_full))])
x_train = (train.review) 
y_train = (train.sentiment)
x_validate = (validate.review)
y_validate = (validate.sentiment)
x_test = (test.review )
y_test = (test.sentiment)
maxLen = 1000#len(max(X_Y_full.review, key=len).split(' '))#  maximum length of sequences


50000


In [101]:
#Tokenise the sequences and give indices to them
from keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=1000,filters='![0-9]"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=False)
tokenizer.fit_on_texts(X_Y_full['review'])

In [102]:
#Tokenise train test and validation
unique_vocab = len(tokenizer.word_index)
unique_vocab
x_train = tokenizer.texts_to_sequences(x_train)
#print(x_train[0])
x_train = sequence.pad_sequences(x_train, maxlen=maxLen)
x_test = tokenizer.texts_to_sequences(x_test)
x_test = sequence.pad_sequences(x_test, maxlen=maxLen)
x_validate = tokenizer.texts_to_sequences(x_validate)
x_validate = sequence.pad_sequences(x_validate, maxlen=maxLen)

155554

In [105]:
#Convert target to 0 and 1 and do numpy transformations to leverage fast parallel processing
y_train_new = ps.sqldf("select case when sentiment='positive' then 1 else 0 end as sentiment_new from train")
y_test_new = ps.sqldf("select case when sentiment='positive' then 1 else 0 end as sentiment_new from test")
y_validation_new = ps.sqldf("select case when sentiment='positive' then 1 else 0 end as sentiment_new from validate")
y_train_new = np.asarray(y_train_new['sentiment_new'])
y_test_new = np.asarray(y_test_new['sentiment_new'])
y_validation_new = np.asarray(y_validation_new['sentiment_new'])
x_train = np.asarray(x_train)
y_train_new = np.asarray(y_train_new)
x_validate = np.asarray(x_validate)
y_validation_new = np.asarray(y_validation_new)
x_test = np.asarray(x_test)
y_test_new = np.asarray(y_test_new)

In [217]:
# model 1 - Simple RNN
model = Sequential()
embedding_size=50
model.add(Embedding(1000,
                    embedding_size,
                    input_length=maxLen,batch_input_shape=[500, None]))
model.add(layers.SimpleRNN(100,return_sequences=False))
model.add(Dense(100,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (500, None, 50)           50000     
_________________________________________________________________
lstm_24 (LSTM)               (500, None, 100)          60400     
_________________________________________________________________
lstm_25 (LSTM)               (500, 100)                80400     
_________________________________________________________________
dense_65 (Dense)             (500, 100)                10100     
_________________________________________________________________
dropout_25 (Dropout)         (500, 100)                0         
_________________________________________________________________
dense_66 (Dense)             (500, 1)                  101       
Total params: 201,001
Trainable params: 201,001
Non-trainable params: 0
_______________________________________________

In [218]:
#Loss Function # tried different values for learning_rate
from tensorflow.keras import optimizers
AD = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=AD,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [219]:
#Checkpointing
import os
checkpoint_dir = 'bestt_2_1'
import shutil
try:
    shutil.rmtree(checkpoint_dir)
except:
    print("directory not used yet.")

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_")

cb=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='loss',
    save_weights_only=True,
    save_best_only=True)

directory not used yet.


In [220]:
#Early stopping # Patience values experimented for different values
from tensorflow.keras.callbacks import EarlyStopping 
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=7) 

In [ ]:
#Training SimpleRNN
hist = model.fit(x_train, y_train_new,
          batch_size=500,
          epochs=100,callbacks=[cb],verbose=1,validation_data=(x_validate, y_validation_new)
          )
model.save('Part_A_Best_2_50k_SimpleRNN.h5')  

In [229]:
#Model 1 Testing with custom examples so batch size as 1
model = Sequential()
embedding_size=50
model.add(Embedding(1000,
                    embedding_size,
                    input_length=maxLen,batch_input_shape=[1, None]))
model.add(layers.SimpleRNN(100,return_sequences=False))
model.add(Dense(100,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(1,activation='sigmoid'))
model.load_weights(tf.train.latest_checkpoint('bestt_2_1'))
model.build(tf.TensorShape([1, None]))
#custom examples
Xnew = np.array(["This movie should have ended as soon as the joke about Bebe's Kids is told in the opening. I liked Robin Harris and most of his comedy but the really funny routines were not meant to be something the whole family could go see. Liken it to taking the point of one of Jerry Seinfeld's jokes and attempting to squeeze the joke for as long as you can in order to turn it into a movie. This movie had to be self-serving because I can not find anyone who found the settings or antics familiar. Whats most funny about this movie is the gumption of the writers and producers to pass it off as something of value. 1/10 stars because there is no half star."])
Xnew = np.array(["poor expectation but good result"])
X1new_indices = tokenizer.texts_to_sequences(Xnew)
X1new_indices = sequence.pad_sequences(X1new_indices, maxlen=maxLen)
ynew = model.predict_classes(X1new_indices,batch_size=1)
ynew[0][0]

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (1, None, 50)             50000     
_________________________________________________________________
lstm_28 (LSTM)               (1, None, 100)            60400     
_________________________________________________________________
lstm_29 (LSTM)               (1, 100)                  80400     
_________________________________________________________________
dense_69 (Dense)             (1, 100)                  10100     
_________________________________________________________________
dropout_27 (Dropout)         (1, 100)                  0         
_________________________________________________________________
dense_70 (Dense)             (1, 1)                    101       
Total params: 201,001
Trainable params: 201,001
Non-trainable params: 0
_______________________________________________

In [240]:
#checking and testing load_model functionality
import tensorflow as tf 
classifierLoad = tf.keras.models.load_model('Part_A_Best_2_50k.h5')
loss, acc = classifierLoad.evaluate(x_test, y_test_new)
loss, acc = classifierLoad.evaluate(x_train, y_train_new)
print("Test accuracy = ", acc)
loss, acc = classifierLoad.evaluate(x_validate, y_validation_new)
print("Test accuracy = ", acc)

In [ ]:
# model 2 - 1 LSTM
model = Sequential()
embedding_size=50
model.add(Embedding(1000,
                    embedding_size,
                    input_length=maxLen,batch_input_shape=[500, None]))
model.add(tf.keras.layers.LSTM(100,return_sequences=False, stateful=False,recurrent_initializer='glorot_uniform'))
model.add(Dense(100,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
#Loss Function # tried different values for learning_rate
from tensorflow.keras import optimizers
AD = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=AD,
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
#Checkpointing
import os
checkpoint_dir = 'bestt_2_simpleRNN'
import shutil
try:
    shutil.rmtree(checkpoint_dir)
except:
    print("directory not used yet.")

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_")

cb=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='loss',
    save_weights_only=True,
    save_best_only=True)


In [ ]:
#Early stopping # Patience values experimented for different values
from tensorflow.keras.callbacks import EarlyStopping 
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=7) 


In [ ]:
#Training LSTM model
hist = model.fit(x_train, y_train_new,
          batch_size=500,
          epochs=100,callbacks=[cb,es],verbose=1,validation_data=(x_validate, y_validation_new)
          )
model.save('Part_A_Best_2_50k_LSTM.h5')  


In [ ]:

#Model 2 Testing with custom examples so batch size as 1
model = Sequential()
embedding_size=50
model.add(Embedding(1000,
                    embedding_size,
                    input_length=maxLen,batch_input_shape=[1, None]))
model.add(tf.keras.layers.LSTM(100,return_sequences=False, stateful=True,recurrent_initializer='glorot_uniform'))
model.add(Dense(100,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(1,activation='sigmoid'))
model.load_weights(tf.train.latest_checkpoint('bestt_2_LSTM'))
model.build(tf.TensorShape([1, None]))
#custom examples
Xnew = np.array(["This movie should have ended as soon as the joke about Bebe's Kids is told in the opening. I liked Robin Harris and most of his comedy but the really funny routines were not meant to be something the whole family could go see. Liken it to taking the point of one of Jerry Seinfeld's jokes and attempting to squeeze the joke for as long as you can in order to turn it into a movie. This movie had to be self-serving because I can not find anyone who found the settings or antics familiar. Whats most funny about this movie is the gumption of the writers and producers to pass it off as something of value. 1/10 stars because there is no half star."])
Xnew = np.array(["poor expectation but good result"])
X1new_indices = tokenizer.texts_to_sequences(Xnew)
X1new_indices = sequence.pad_sequences(X1new_indices, maxlen=maxLen)
ynew = model.predict_classes(X1new_indices,batch_size=1)
ynew[0][0]

In [ ]:
#Model 3 - Stacked LSTM -2 LSTMs
model = Sequential()
embedding_size=50
model.add(Embedding(1000,
                    embedding_size,
                    input_length=maxLen,batch_input_shape=[500, None]))
model.add(tf.keras.layers.LSTM(100,return_sequences=True,  stateful=True,recurrent_initializer='glorot_uniform'))
model.add(tf.keras.layers.LSTM(100,return_sequences=False, stateful=True,recurrent_initializer='glorot_uniform'))
model.add(Dense(100,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
#Loss Function # tried different values for learning_rate
from tensorflow.keras import optimizers
AD = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=AD,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
#Checkpointing
import os
checkpoint_dir = 'bestt_2_2LSTM'
import shutil
try:
    shutil.rmtree(checkpoint_dir)
except:
    print("directory not used yet.")

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_")

cb=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='loss',
    save_weights_only=True,
    save_best_only=True)

In [ ]:
#Early stopping # Patience values experimented for different values
from tensorflow.keras.callbacks import EarlyStopping 
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=7) 


In [ ]:
#Training 2 LSTM model
hist = model.fit(x_train, y_train_new,
          batch_size=500,
          epochs=100,callbacks=[cb,es],verbose=1,validation_data=(x_validate, y_validation_new)
          )
model.save('Part_A_Best_2_50k_2LSTM.h5')  

In [ ]:
#Model 3 Testing with custom examples so batch size as 1
model = Sequential()
embedding_size=50
model.add(Embedding(1000,
                    embedding_size,
                    input_length=maxLen,batch_input_shape=[1, None]))
model.add(tf.keras.layers.LSTM(100,return_sequences=False, stateful=True,recurrent_initializer='glorot_uniform'))
model.add(tf.keras.layers.LSTM(100,return_sequences=False, stateful=True,recurrent_initializer='glorot_uniform'))
model.add(Dense(100,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(1,activation='sigmoid'))
model.load_weights(tf.train.latest_checkpoint('bestt_2_1'))
model.build(tf.TensorShape([1, None]))
#custom examples
Xnew = np.array(["This movie should have ended as soon as the joke about Bebe's Kids is told in the opening. I liked Robin Harris and most of his comedy but the really funny routines were not meant to be something the whole family could go see. Liken it to taking the point of one of Jerry Seinfeld's jokes and attempting to squeeze the joke for as long as you can in order to turn it into a movie. This movie had to be self-serving because I can not find anyone who found the settings or antics familiar. Whats most funny about this movie is the gumption of the writers and producers to pass it off as something of value. 1/10 stars because there is no half star."])
Xnew = np.array(["poor expectation but good result"])
X1new_indices = tokenizer.texts_to_sequences(Xnew)
X1new_indices = sequence.pad_sequences(X1new_indices, maxlen=maxLen)
ynew = model.predict_classes(X1new_indices,batch_size=1)
ynew[0][0]

In [ ]:
####################################Pretrained Embeddings#############################

In [96]:
#Shaping the embedding vector from the file
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [97]:
#MAking Text<->index <-> Vector pairs
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')

In [98]:
# Custom created pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    
    
    vocab_len = len(word_to_index) + 1                  
    emb_dim = word_to_vec_map["movie"].shape[0]     
    emb_matrix = np.full([1000, emb_dim], 0, dtype = float) 
    for word, idx in word_to_index.items():
        if(idx<1000):
            emb_matrix[idx, :] = word_to_vec_map[word]
    embedding_layer = Embedding(1000,emb_dim,weights=[emb_matrix],trainable=False)
    embedding_layer.build((None,)) 
    return embedding_layer
embeddings = pretrained_embedding_layer(word_to_vec_map, word_to_index)

In [130]:
#Importing Extra modules
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation,SimpleRNN,concatenate,Flatten,MaxPooling1D
from tensorflow.keras.models import Model


In [101]:
#Model for the pretrained embeddings and the rest of the architecture
def pretrained_moedl(input_shape, word_to_vec_map, word_to_index):
   
    raw_inputs = Input(shape=input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(raw_inputs)   
    X = SimpleRNN(units = 100)(embeddings)
    Y = Dropout(rate = 0.5 )(X)
    B = Dense(units = 100)(Y)
    B = Dense(units = 1)(B)
    C = Activation("sigmoid")(B)
    model = Model(inputs=raw_inputs, outputs=C)
    return model
model_pre = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model_pre.summary()

In [74]:
#Loss function
model_pre.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [75]:
#checkpointing
import os
checkpoint_dir = 'trained embed'
import shutil
try:
    shutil.rmtree(checkpoint_dir)
except:
    print("directory not used yet.")

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_")

cb=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='loss',
    save_weights_only=True,
    save_best_only=True)

directory not used yet.


In [ ]:
#model training
model_pre.fit(x_train, y_train_new, epochs = 50, batch_size = 1000, callbacks=[cb,es],shuffle=True)

In [68]:
#Model Evaluation
loss, acc = model_pre.evaluate(x_test, y_test_new)
#print()
print("Test accuracy = ", acc)

loss, acc = model_pre.evaluate(x_train, y_train_new)
#print()
print("Test accuracy = ", acc)

loss, acc = model_pre.evaluate(x_validate, y_validation_new)
#print()
print("Test accuracy = ", acc)

4000/4000 [==============================] - 8s 2ms/sample - loss: 1.0732 - acc: 0.6545
Test accuracy =  0.6545
10000/10000 [==============================] - 20s 2ms/sample - loss: 0.1804 - acc: 0.9302
Test accuracy =  0.9302
6000/6000 [==============================] - 12s 2ms/sample - loss: 1.0544 - acc: 0.6603
Test accuracy =  0.66033334


In [143]:
#CNN - as a replacement for LSTM
raw_inputs = Input(shape=(1000,), dtype='int32')
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
embeddings = embedding_layer(raw_inputs)  
#heterogeneous kernel sized convolution layers

x1 = Conv1D(100,
                        300,
                        padding='same',
                        activation='relu',
                        strides=1)(embeddings) 
x4=MaxPooling1D()(x1)

raw_inputs1 = Input(shape=(1000,), dtype='int32')
embedding_layer1 = pretrained_embedding_layer(word_to_vec_map, word_to_index)
embeddings1 = embedding_layer1(raw_inputs1) 
#Experimented with 3 5 & 7 kernel sizes also
x2 = Conv1D(100,
                        600,
                        padding='same',
                        activation='relu',
                        strides=1)(embeddings1) 
x3 = MaxPooling1D()(x2)

raw_inputs2 = Input(shape=(1000,), dtype='int32')
embedding_layer2 = pretrained_embedding_layer(word_to_vec_map, word_to_index)
embeddings2 = embedding_layer2(raw_inputs2) 

x5 = Conv1D(100,
                        900,
                        padding='same',
                        activation='relu',
                        strides=1)(embeddings2) 
x7 = MaxPooling1D()(x5)


summ = concatenate([x4,x3,x7])
f = Flatten()(summ)
#L = LSTM(50, return_sequences=False)(summ)
DR = Dropout(rate=0.5)(f)
B = Dense(units = 1)(DR)
    # Add a softmax activation
C = Activation("sigmoid")(B)

model_con = Model(inputs = [raw_inputs,raw_inputs1,raw_inputs2],outputs = C)
model_con.summary()



Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           [(None, 1000)]       0                                            
__________________________________________________________________________________________________
input_45 (InputLayer)           [(None, 1000)]       0                                            
__________________________________________________________________________________________________
input_46 (InputLayer)           [(None, 1000)]       0                                            
__________________________________________________________________________________________________
embedding_50 (Embedding)        (None, 1000, 50)     50000       input_44[0][0]                   
___________________________________________________________________________________________

In [ ]:
#Loss function & train
model_con.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_con.fit([x_train,x_train,x_train], y_train_new,batch_size = 1000, epochs = 50, shuffle=True,validation_data=([x_validate,x_validate,x_validate], y_validation_new))

Train on 10000 samples, validate on 6000 samples
Epoch 1/50
10000/10000 [==============================] - 468s 47ms/sample - loss: 0.7285 - acc: 0.5010 - val_loss: 0.6928 - val_acc: 0.5210
Epoch 2/50
10000/10000 [==============================] - 466s 47ms/sample - loss: 0.6975 - acc: 0.5322 - val_loss: 0.6894 - val_acc: 0.5367
Epoch 3/50
 9000/10000 [==========================>...] - ETA: 34s - loss: 0.6786 - acc: 0.5662 

In [ ]:
#CNN before LSTM
raw_inputs = Input(shape=(1000,), dtype='int32')
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
embeddings = embedding_layer(raw_inputs)  


x1 = Conv1D(100,
                        300,
                        padding='same',
                        activation='relu',
                        strides=1)(embeddings) 
x4=MaxPooling1D()(x1)

raw_inputs1 = Input(shape=(1000,), dtype='int32')
embedding_layer1 = pretrained_embedding_layer(word_to_vec_map, word_to_index)
embeddings1 = embedding_layer1(raw_inputs1) 
#Experimented with 3 5 & 7 kernel sizes also
x2 = Conv1D(100,
                        600,
                        padding='same',
                        activation='relu',
                        strides=1)(embeddings1) 
x3 = MaxPooling1D()(x2)

raw_inputs2 = Input(shape=(1000,), dtype='int32')
embedding_layer2 = pretrained_embedding_layer(word_to_vec_map, word_to_index)
embeddings2 = embedding_layer2(raw_inputs2) 

x5 = Conv1D(100,
                        900,
                        padding='same',
                        activation='relu',
                        strides=1)(embeddings2) 
x7 = MaxPooling1D()(x5)


summ = concatenate([x4,x3,x7])
#f = Flatten()(summ)
L = LSTM(50, return_sequences=False)(summ)
DR = Dropout(rate=0.5)(L)
B = Dense(units = 1)(DR)
    # Add a softmax activation
C = Activation("sigmoid")(B)
#summ = keras.layers.add([x1, x2]) 
model_con = Model(inputs = [raw_inputs,raw_inputs1,raw_inputs2],outputs = C)
model_con.summary()

#Loss function & train
model_con.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_con.fit([x_train,x_train,x_train], y_train_new,batch_size = 1000, epochs = 50, shuffle=True,validation_data=([x_validate,x_validate,x_validate], y_validation_new))

#model saving
model_con.save('conv_for_LSTM.h5')